## Kotlin: What and Why

In [1]:
data class Person( // データクラス
    val name: String, // 読み込み専用のプロパティ
    val age: Int? = null // ヌル許容型　引数のデフォルト値
)

fun main() {
    val person = listOf(
        Person("Alice", age = 29), // Named argument
        Person("Bob"), // Trailing comma
    )
    val oldest = person.maxBy {
        it.age ?: 0 // null-coalescing Elvis operator
    }
    println("The oldest is: $oldest")
}

main()

The oldest is: Person(name=Alice, age=29)


簡単なまとめ
- Bobの`age`はnullなのでデフォルト値が入っている
- `?:`はElvis演算子と呼ばれる。Bobの値は`null`かどうか確かめて、`null`なので0にする

Kotlinの特徴
- 静的型付け（コンパイル時に意味のあるエラーをいっぱい出してくれる）
- OOPかつ関数型に対応（エレガントなコードかつ高度な抽象化）
- 非同期処理

Kotlinの最初期のゴール
- 簡単
- 生産的
- 安全なJavaの代替

静的型付けの利益
- パフォーマンス：メソッドの呼び出しが高速。なぜなら、ランタイム時にどのようなメソッドが呼ばれるかを決めずに済むので
- 信頼性：コンパイラが型を使ってプログラムの整合性を識別する。よってランタイムにクラッシュすることを避けやすい
- 保守性：不慣れなコードであっても、そのコードがどのような型を扱っているかが分かるため、理解しながら作業しやすくなる
- ツールのサポート：信頼のあるレファクタリングを可能にし、正確なコードの補完や他IDEの恩恵を得られる

- とにかくランタイムのエラーを最悪と考えると動的言語（Python、JavaScript）には大きな問題がある

- Kotlinは多くの場合、文脈から型を推測できる


関数型に対して第一級のサポートを持つ。これには関数型プログラミングと、どのようにKotlinの中でサポートされているのかを後ほど確認する。

Kotlin = OOP + FP

FPの概念：
- 第一級関数 (First-class functions)：関数を値のように扱う。関数を変数に入れることができる。他の変数に代入することができるし、関数に返り値にもできる。
- イミュータブル：イミュータブルなオブジェクトで動かす。これはオブジェクトを生成した後で状態が変化しない
- 副作用がない：純粋関数を書く。関数はいつでも同じ入力に対して同じ出力を返す。

```kotlin
messages
    .filter { it.body.isNotBlanck() && !it.isRead }
    .map(Message::sender)
    .distinct()
    .sortedBy(sender::name)
```

並列の問題と非同期プログラミングのために、Kotlinはコルーチンと呼ばれる*suspendable computation*（中断可能な計算）を使ってアプローチする
`

```kotlin
suspend fun processUser(credentials: Credentials) {
    val user = authenticate(credentials)    // 結構長い処理
    val data = loadUserData(user)           // トップダウンで
    val profilePicutre = loadImage(data.imageID) // アプリをブロックしない
}

suspend fun authenticate(c: Credentials): User { /* */ }
suspend fun loadUserData(u: User): Data { /* */ }
suspend fun loadImage(id: Int): Image { /* */ }
```

ネットワークの呼び出しは任意の長さをとるだろう。それぞれのネットワークリクエストを実行する時、`processUser`の実行は結果のために中断される。しかし、このコードのスレッドは走ったままでありブロックされない。一方で`processUser`の結果は待っており、その頃は他のタスクが可能となる。例えばユーザ入力のレスポンスなど。

```kotlin
suspend fun loadAndOverlay(first: String, second: String): Image =
    coroutineScope {
        val firstDeferred = async { loadImage(first) }
        val secondDeferred = async { loadImage(second) }
        combineImage(firstDeferred.await(), secondDeferred.await())
}
```